In [1]:
import math
import numpy as np
import pandas as pd
import yfinance as yf
from yahoo_fin import options
import datetime as dt
import statsmodels.api as sm
import matplotlib.pyplot as plt
from scipy.stats import norm
from numpy.linalg import inv
from scipy.stats import ncx2
from scipy.optimize import fsolve
from scipy.special import comb
from scipy.stats import binom

# Uniform distribution

In [2]:
# LGM uniform distribution
def lgm_uniform(n,x0):
    m = (2**31)-1
    a = 7**5
    b = 0
    lgm0 = [] # original random numbers
    lgm0.append(x0) # set x_0 
    lgm = [] # random numbers in [0,1]
    lgm.append(1/m)
    for i in range(1,n):
        lgm0.append((a*lgm0[i-1]+b) % m)
        lgm.append(lgm0[i]/m)
    return lgm

# Normal distribution

In [3]:
# Box- Muller Method  
def BoxMuller(u1,u2,n):
    z1 = []
    z2 = []
    for i in range(0,n):
        z1.append(np.sqrt(-2*np.log(u1[i]))*math.cos(2*(math.pi)*u2[i]))
        z2.append(np.sqrt(-2*np.log(u1[i]))*math.sin(2*(math.pi)*u2[i]))
    return z1,z2

In [4]:
# Polar-Marsaglia method
# As in this method not every u will be used, so we should generate more random numbers
def PolarMarsaglia(u1,u2,n,N):
    # n:length of the uniform distribution
    # N:length of the normal distribution we want
    z1 = []
    z2 = []
    for i in range(0,n):
        v1 = 2*u1[i]-1
        v2 = 2*u2[i]-1
        w = (v1**2) + (v2**2)
        if w <= 1:
            z1.append(v1*np.sqrt((-2)*np.log(w)/w))
            z2.append(v2*np.sqrt((-2)*np.log(w)/w))
        if len(z1) == N:
            break
    return z1,z2

# Black-scholes

In [36]:
def BS_price(S, K, T, r, sigma, div, x):
    # x=1 means call option, x=0 means put option
    d1 = (np.log(S/K) + (r + (0.5*sigma**2) - div)*T)/(sigma*np.sqrt(T))
    d2 = (np.log(S/K) + (r - (0.5*sigma**2) - div)*T)/(sigma * np.sqrt(T))
    p = (2*x-1)*S*np.exp((-1)*div*T)*norm.cdf((2*x-1)*d1, 0.0, 1.0) - (2*x-1)*K*np.exp(-r*T)*norm.cdf((2*x-1)*d2, 0.0, 1.0)
    return p

# Monte Carlo simulation: European option

In [7]:
# simulate 2N stock paths
def stock_path(S0,T,r,sigma,dt,N):
    n = int(round(T/dt)) # the number of dt in the whole life
    dw = np.sqrt(dt) * np.random.normal(0,1,n*N).reshape(N,n)
    dS1 = 1 + r*dt + sigma*dw
    dS2 = 1 + r*dt - sigma*dw
    path1 = pd.DataFrame(S0 * np.cumprod(dS1, axis = 1))
    path2 = pd.DataFrame(S0 * np.cumprod(dS2, axis = 1))
    path = pd.concat([path1,path2]).reset_index(drop = True)
    return path

In [19]:
def MT_European(S0,T,r,sigma,N,X,cp):
    
    # cp=1:call, cp=-1:put
    wT = np.random.normal(0,np.sqrt(T),N)
    ST = S0 * np.exp(sigma * wT + (r - 0.5*(sigma**2))*T)
    
    # calculate payoff
    payoff = []
    for i in range(0,N):
        payoff.append(max(cp*(ST[i]-X),0))
        
    # calculate price
    price = np.mean(payoff) * np.exp(-1*r*T)
    return price

# Estimate pi using importance sampling

I used Acceptance-Rejection Method to generate a series of Y that following the distribution with PDF t(y): $$ t(y) = \frac{1-ay^2}{1-a/3},\ 0\leq y\leq1;\ t(y) = 0,\ otherwise$$ Then calculate the expectation of $\frac{4\sqrt{1-y_i^2}}{t(y_i)}$ to get the estimation of $\pi$. It comes out that the mean is closer to the real number and the variance decreases dramatically, which means this estimation is much more precise than previous one. The error may be even smaller if I increase the number of Y.

In [17]:
a = 0.74

# Create a series of y follwing t(y) distribution
# Acceptance-Rejection Method
U = np.random.uniform(0,1,1000000)
V = np.random.uniform(0,1,1000000) 
M = 1/(1-a/3) # max value of t(y)
Y = []
for i in range(0,len(U)):
    if U[i]<(1/M)*(1 - a*(V[i])**2)/(1-a/3):
        Y.append(V[i])

# estimate pi
Y = np.array(Y)
ty = (1 - a*(Y**2))/(1-a/3) # t(y)
fy_ty = np.sqrt(1-Y**2)/ty # f(y)/t(y)
ip_pi = np.mean(fy_ty) * 4

# Numerical Computation of N(x)

In [ ]:
def NX(x):
    d1 = 0.0498673470
    d2 = 0.0211410061
    d3 = 0.0032776263
    d4 = 0.0000380036
    d5 = 0.0000488906
    d6 = 0.0000053830
    if x > 0:
        N_x = 1 - 0.5*((1+d1*x+d2*(x**2)+d3*(x**3)+\
                        d4*(x**4)+d5*(x**5)+d6*(x**6))**(-16))
    else:
        N_x = 0.5*((1+d1*(-1*x)+d2*((-1*x)**2)+d3*((-1*x)**3)+\
                    d4*((-1*x)**4)+d5*((-1*x)**5)+d6*((-1*x)**6))**(-16))
    return N_x

# Heston Model: estimate S(T)

$$ S_{t+dt} = S_t + rS_tdt + \sqrt{V_t}S_t\sqrt{dt}Z_t^1(0,1) $$

$$ V_{t+dt} = V_t + \alpha(\beta - V_t)dt + \sigma\sqrt{V_t}\sqrt{dt}(\rho Z_t^1(0,1)+\sqrt{1-\rho^2}Z_t^2(0,1)$$

In [ ]:
# Full Truncation
def ST_Full(S0,V0,r,dt,T,alpha,beta,sigma,rho):
    S = S0
    V = V0
    for i in range(0,round(T/dt)):
        z1 = np.random.normal(0,1,1)
        z2 = np.random.normal(0,1,1)
        S = S + r*S*dt + np.sqrt(max(0,V)*dt)*S*z1
        V = V + alpha*(beta-max(0,V))*dt +\
            sigma*np.sqrt(max(0,V)*dt)*(rho*z1+np.sqrt(1-rho**2)*z2)
    return S

In [ ]:
# Partial Truncation
def ST_Part(S0,V0,r,dt,T,alpha,beta,sigma,rho):
    S = S0
    V = V0
    for i in range(0,round(T/dt)):
        z1 = np.random.normal(0,1,1)
        z2 = np.random.normal(0,1,1)
        S = S + r*S*dt + np.sqrt(max(0,V)*dt)*S*z1
        V = V + alpha*(beta-V)*dt +\
            sigma*np.sqrt(max(0,V)*dt)*(rho*z1+np.sqrt(1-rho**2)*z2)
    return S

In [ ]:
# Reflection methods
def ST_Ref(S0,V0,r,dt,T,alpha,beta,sigma,rho):
    S = S0
    V = V0
    for i in range(0,round(T/dt)):
        z1 = np.random.normal(0,1,1)
        z2 = np.random.normal(0,1,1)
        S = S + r*S*dt + np.sqrt(abs(V)*dt)*S*z1
        V = abs(V) + alpha*(beta-abs(V))*dt +\
            sigma*np.sqrt(abs(V)*dt)*(rho*z1+np.sqrt(1-rho**2)*z2)
    return S

# Halton's Low-Discrepancy Sequences

In [23]:
def van_der_corput(n_sample, base=2):
    sequence = []
    for i in range(n_sample):
        n_th_number, denom = 0., 1.
        while i > 0:
            i, remainder = divmod(i, base)
            denom *= base
            n_th_number += remainder / denom
        sequence.append(n_th_number)
    return sequence

def halton(dim, n_sample, base):
    sample = [van_der_corput(n_sample + 1, dim) for dim in base]
    sample = np.stack(sample, axis=-1)[1:]
    return sample

In [ ]:
# Use Halton’s Low-Discrepancy Sequences to price European Call options.
def HaltonEurCall(S0,X,T,r,sigma,N,b1,b2):
    
    H1 = halton(2, N, [b1,b2])[:,0]
    H2 = halton(2, N, [b1,b2])[:,1]
    
    # Box- Muller Method    
    z1 = []
    z2 = []
    payoff = []
    for i in range(0,N):
        z1.append(np.sqrt(-2*np.log(H1[i]))*math.cos(2*(math.pi)*H2[i]))
        z2.append(np.sqrt(-2*np.log(H1[i]))*math.sin(2*(math.pi)*H2[i]))
        
        payoff.append(max(0,S0*np.exp((r-0.5*sigma**2)*T+sigma*np.sqrt(T)*z1[i])-X))
        payoff.append(max(0,S0*np.exp((r-0.5*sigma**2)*T+sigma*np.sqrt(T)*z2[i])-X))
    
    price = np.mean(payoff) * np.exp(-r*T)
    
    return price

# Binomial Method

In [ ]:
# Eurpean call
def BinomialEurCall(n,u,d,p,S0,X,T):
    E_payoff = 0
    for i in range(0,n+1):
        E_payoff = E_payoff + binom.pmf(i,n,p) * max(0,(S0*(u**i)*(d**(n-i))-X))
    bino_p = E_payoff * np.exp(-r*T) 
    return bino_p    

In [ ]:
# American call
def BinomialAmerCall(r,n,T,s0,K,sigma):
    
    # u,d,p
    delta = T/n
    u = np.exp(delta*(r-0.5*(sigma**2)) + sigma*np.sqrt(delta))
    d = np.exp(delta*(r-0.5*(sigma**2)) - sigma*np.sqrt(delta))
    p = 0.5
    
    # create stock price paths dataframe
    x = np.linspace(0, n, n+1)
    mat_u = np.tile(x,(n+1,1))
    mat_d = np.triu(mat_u.T)
    mat_ud = np.triu(mat_u - mat_d)
    p_path = np.triu(s0 * np.power(u,mat_ud) *  np.power(d,mat_d))
    p_path = pd.DataFrame(p_path)
    
    v_path = p_path.copy() # value paths dataframe
    v_path[n] = v_path[n].apply(lambda x: max(0,x-K)) # payoff at T

    for i in np.linspace(n-1,0,n):
        i = int(i)
        for m in np.linspace(0,i,i+1):
            m = int(m)
            EV = max(0,v_path.iloc[m,i]-K)
            CV = (p*v_path.iloc[m,i+1] + (1-p)*v_path.iloc[m+1,i+1])*np.exp(-delta*r)
            if EV > CV:
                v_path.iloc[m,i] = EV
            else:
                v_path.iloc[m,i] = CV

    price = v_path.iloc[0,0]
    
    return price

In [ ]:
# American put
def BinomialAmerPut(r,n,T,S0,k,sigma):
    
    # u,d,p
    delta = T/n
    u = np.exp(delta*(r-0.5*(sigma**2)) + sigma*np.sqrt(delta))
    d = np.exp(delta*(r-0.5*(sigma**2)) - sigma*np.sqrt(delta))
    p = 0.5
    
    # create stock price paths dataframe
    x = np.linspace(0, n, n+1)
    mat_u = np.tile(x,(n+1,1))
    mat_d = np.triu(mat_u.T)
    mat_ud = np.triu(mat_u - mat_d)
    p_path = np.triu(S0 * np.power(u,mat_ud) *  np.power(d,mat_d))
    p_path = pd.DataFrame(p_path)
    
    v_path = p_path.copy() # value paths dataframe
    v_path[n] = v_path[n].apply(lambda x: max(0,k-x)) # payoff at maturity

    for i in np.linspace(n-1,0,n):
        i = int(i)
        for m in np.linspace(0,i,i+1):
            m = int(m)
            EV = max(0, k-v_path.iloc[m,i])
            CV = (p*v_path.iloc[m,i+1] + (1-p)*v_path.iloc[m+1,i+1])*np.exp(-delta*r)
            if EV > CV:
                v_path.iloc[m,i] = EV
            else:
                v_path.iloc[m,i] = CV

    price = v_path.iloc[0,0]
    
    return price

In [ ]:
# JR binomial method
def BinomialJR(n,sigma,r,S0,X,T):
    delta = T/n
    u = np.exp(delta*(r-0.5*(sigma**2)) + sigma*np.sqrt(delta))
    d = np.exp(delta*(r-0.5*(sigma**2)) - sigma*np.sqrt(delta))
    p = 0.5
    E_payoff = 0
    for i in range(0,n+1):
        E_payoff = E_payoff + binom.pmf(i,n,p) * max(0,(S0*(u**i)*(d**(n-i))-X))
    JR_p = E_payoff * np.exp(-r*T) 
    return JR_p   

# Trinomial method

$$Price = e^{-rT} \sum_{i=0}^n\sum_{k=0}^{n-i}\tbinom{n}{i}p_u^i\tbinom{n-i}{k}p_d^kp_m^{n-k-i}(S_0u^id^k-X)^+
$$

In [ ]:
# European Call
def TrinomialEurCall(n,u,d,pu,pm,pd,T,r,sigma,S0,X):
    E_payoff = 0
    for i in range(0,n+1):
        for k in range(0,n-i+1):
            E_payoff = E_payoff + comb(n,i)*comb(n-i,k)*(pu**i)*(pd**k)*(pm**(n-k-i))*max(0,S0*(u**i)*(d**k)-X)
    tri_p = E_payoff * np.exp(-r*T) 
    return tri_p

# LSMC

In [30]:
# function outputs continuation value based on monomials reg
def monomials_reg(dsc_payoff,spot,order):
    
    #initialise object with RHS of reg
    X=np.empty(len(dsc_payoff)*order)
    X.shape=((len(dsc_payoff),order))
    
    #compute X based on order
    for j in range(order):
        X[:,j]=spot**j
    
    #fit regression
    Exp_Y=sm.OLS(dsc_payoff,X).fit()
    
    return Exp_Y.get_prediction().predicted_mean

In [31]:
# function outputs continuation value based on hermite polynomials reg
def hermite_reg(dsc_payoff,spot,order):
    
    #initialise object with RHS of reg
    X=np.empty(len(dsc_payoff)*order)
    X.shape=((len(dsc_payoff),order))
    
    #compute X based on order
    for j in range(order):
        if j==0:
            X[:,j]=spot**j
        elif j==1:
            X[:,j]=spot*2
        elif j==2:
            X[:,j]=4*spot**2-2
        elif j==3:
            X[:,j]=8*spot**3-12*spot
    
    #fit regression
    Exp_Y=sm.OLS(dsc_payoff,X).fit()
    
    return Exp_Y.get_prediction().predicted_mean

In [32]:
def laguerre_reg(dsc_payoff,spot,order):
    
    #initialise object with RHS of reg
    X=np.empty(len(dsc_payoff)*order)
    X.shape=((len(dsc_payoff),order))
    
    #compute X based on order
    for j in range(order):
        if j==0:
            X[:,j]=np.exp(-spot/2)
        elif j==1:
            X[:,j]=np.exp(-spot/2)*(1-spot)
        elif j==2:
            X[:,j]=np.exp(-spot/2)*(1-2*spot+0.5*spot**2)
        elif j==3:
            X[:,j]=np.exp(-spot/2)*(1-3*spot+3*spot+1.5*spot**2-(1/6)*spot**3)
    
    #fit regression
    Exp_Y=sm.OLS(dsc_payoff,X).fit()
    
    return Exp_Y.get_prediction().predicted_mean

In [41]:
#define the generic longstaff-shwartz function
def LSMC(S_0,r,sigma,K,T,n_sim,n_steps,call,poly,order,seed):
    
    #numpy set seed
    np.random.seed(seed)
    
    #compute time-step
    h=T/n_steps
    
    #initialise object containing paths
    S=np.empty(n_sim*n_steps)
    S.shape=((n_sim,n_steps))
    
    #set time-0 value
    S[:,0]=S_0*np.ones(n_sim)
    
    #compute spot paths
    for t in range(1,n_steps):
        Z=np.random.normal(0,1,int(n_sim/2))
        S[:int(n_sim/2),t]=S[:int(n_sim/2),t-1]*np.exp((r-0.5*sigma**2)*h+sigma*np.sqrt(h)*Z)
        S[int(n_sim/2):,t]=S[int(n_sim/2):,t-1]*np.exp((r-0.5*sigma**2)*h-sigma*np.sqrt(h)*Z)
    
    #initialise array for results
    discounted_payoffs=np.zeros(n_sim)
    #start backward induction
    for i in range(len(S[0])-1,0,-1):
        
        #compute payoff 
        if call==1:
            intrinsic_value=np.maximum(S[:,i]-K,0)
            
        else:
            intrinsic_value=np.maximum(K-S[:,i],0)

        #for the final payoff there is no continuation
        if i==len(S[0])-1:

            #discount final payoff to t-1
            discounted_payoffs=intrinsic_value*math.exp(-r*h)

        #compute value from continuation
        else:

            #select paths that are ITM
            ITM_index=intrinsic_value>0
            
            #check if there are enough ITM paths
            if np.count_nonzero(ITM_index)>3:
                
                #discounted future payoff
                Y=discounted_payoffs[ITM_index]
                
                #stock prices
                X=S[ITM_index,i]/S[ITM_index,0]
                
                #perform regression
                if poly==0:
                    #use monomials
                    CV=monomials_reg(Y, X, order)
                elif poly==1:
                    #use hermite polys
                    CV=hermite_reg(Y, X, order)
                else:
                    #use laguerre polys
                    CV=laguerre_reg(Y, X, order)
                    
                #get exercise indices
                exercise_index=CV<intrinsic_value[ITM_index]

                #select appropriate indices to update
                ITM_and_exercised=np.where(ITM_index==True)[0][np.where(exercise_index==True)]

                #check if there are paths where exercise occurs
                if len(ITM_and_exercised)>0:
                    
                    #update discounted payoffs where appropriate
                    discounted_payoffs[ITM_and_exercised]=intrinsic_value[ITM_index][exercise_index]                
                
            #discount payoffs to t-1
            discounted_payoffs=math.exp(-r*h)*discounted_payoffs
            
    return np.mean(discounted_payoffs)

In [42]:
LSMC(40,0.06,0.2,40,0.5,100000,int(np.sqrt(100000)),False,2,3,123)

1.7826890757092326

In [ ]:
def LP(k,x):
    formula = np.array([np.exp(-x/2), 
                        (1-x)*np.exp(-x/2), 
                        (1-2*x+(x**2)/2)*np.exp(-x/2), 
                        (1-3*x+(3/2)*(x**2)-(x**3)/6)*np.exp(-x/2)])
    formula = formula[:k]
    return formula

In [ ]:
def LSMC_LP(path, X, T, dt, k, r, N):
    
    n = int(round(T/dt))
    ind = pd.DataFrame(columns = path.columns, index = path.index)
    
    # the last column
    EV = X - path[n-1]  # X-S
    ex = (EV > 0)
    ind[n-1][ex] = 1
    ind[n-1][~ex] = 0
    
    # 0~n-2 columns
    for j in np.linspace(n-2,0,n-1): 
        
        j = int(j)
        ind[j] = 0
        
        # exercise value
        EV = X-path[j] # X-S
        itm = (EV>=0) # in the money rows
        EV = EV[itm] # itm (X-S)
        
        # X,Y for the column
        itm_price = path[j][itm] # in the money stock price (X)
        future_pf = ind.iloc[:,(j+1):]*(X - path.iloc[:,(j+1):]) # future payoffs 
        dc_pf = future_pf * np.exp(-1*(future_pf.columns-j)*dt*r).values 
        Y = dc_pf.sum(axis=1)[itm]
        Y[Y <= 0] = 0 
        Y = np.array(Y).reshape(sum(itm),1) # Y
        
        # estimate coefficients (a)
        L = LP(k,itm_price) # L*X
        A = np.dot(L, L.T)
        b = np.dot(L, Y)
        a = np.dot(inv(A),b)
        
        # expected continuing values
        EV = pd.DataFrame(EV)
        a = np.array(a).reshape(k,1)
        EV['ECV'] = np.multiply(a,L).sum(axis=0)
        
        # make the choice and fill ind matrix
        ex = (EV[j] > EV['ECV']) # the rows that should exercise
        ind.iloc[ex.index,j] = 1
        ind.iloc[ex.index,j+1] = 0

    values = ind*(X - path)  # (X-S)
    dc_pf = values * np.exp(-1*(values.columns+1)*dt*r).values 
    N_pfs = dc_pf.sum(axis=1) # payoffs in all the paths
    N_pfs[N_pfs<0] = 0 
    price = sum(N_pfs) / N
    
    del ind
    
    return price

# Fixed Strike Lookback option

In [ ]:
def lookback_call(r,T,S0,X,N,n,sigma):
    path = stock_path(S0,T,r,sigma,n,N)
    max_price = path.max(axis=1)
    payoff = np.where(max_price-X>0, max_price-X, 0)
    call_price = np.exp(-r*T) * np.mean(payoff)
    return call_price

In [ ]:
def lookback_put(r,T,S0,X,N,n,sigma):
    path = stock_path(S0,T,r,sigma,n,N)
    min_price = path.min(axis=1)
    payoff = np.where(X-min_price>0, X-min_price, 0)
    put_price = np.exp(-r*T) * np.mean(payoff)
    return put_price

# Pricing default option using jump-diffusion 

In [ ]:
def v_path(lambda1, T):
    
    # path without jump
    dt = T/col
    dw = np.sqrt(dt) * np.random.normal(0,1,col*row).reshape(row,col)
    dV = 1 + mu*dt + sigma*dw
    path = pd.DataFrame(V0 * np.cumprod(dV, axis = 1))
    
    # add jump
    for i in range(0,row):
        Y = []
        for j in range(0,col):
            Y.append(np.random.exponential(1/lambda1))
            sum_Y = sum(Y)
            if sum_Y <= T:
                jump_time = int(sum_Y/dt)
                path.iloc[i,jump_time:] = path.iloc[i,jump_time:] * (1+gamma)
            else:
                break
                
    return path

In [ ]:
def DefaultOption(lambda1, lambda2, T):
    
    # set variables
    R = r0 + delta*lambda2
    r = R/12
    n = T*12
    PMT = (L0*r)/(1-1/((1+r)**n))
    a = PMT/r
    b = PMT/(r*((1+r)**n))
    c = 1+r
    beta = (e-alpha)/T
    dt = T/col
    
    # paths of V
    path_T = v_path(lambda1, T)
    
    # qt * Lt
    qt = pd.Series(np.arange(dt,T+dt,dt) * beta + alpha)
    Lt = pd.Series(a - b*pow(c,12*np.arange(dt,T+dt,dt)))
    Lq = Lt * qt
    
    # find stopping time for each path
    all_price = []
    all_extime = []

    for i in range(0,row):   
        # Q
        Vi = path_T.iloc[i,:]
        Q = (Lq >= Vi)
        if sum(Q)>0:
            stop_Q_ind = Vi[Q.cumsum() == 1].index.values[0]
        else:
            stop_Q_ind = col
            
        # Nt
        N = np.random.exponential(1/lambda2)
        if N <= T:
            stop_N_ind = int(N/dt)
        else:
            stop_N_ind = col
        
        # get stop time from min(N,Q)
        stop_ind = min(stop_Q_ind, stop_N_ind)
        if stop_ind <= (col-1):
            if stop_ind == stop_Q_ind:
                payoff = max(0, Lt[stop_ind]-e*Vi[stop_ind])
                all_price.append(payoff * np.exp(-r0*dt*(1+stop_ind)))
            else:
                payoff = abs(Lt[stop_ind]-e*Vi[stop_ind])
                all_price.append(payoff * np.exp(-r0*dt*(1+stop_ind)))
            all_extime.append((stop_ind+1)* dt)
        
    price = sum(all_price)/row
    prob = len(all_extime)/row
    ex_time = np.mean(all_extime)
    
    return price, prob, ex_time

# Finite Difference Method

In [4]:
# Explicit Finite-Difference
def EFD_put(sigma,r,delta_t,delta_x,K,T,S0,SN):
    
    pu = delta_t * (sigma**2/(2*(delta_x**2)) + (r-0.5*(sigma**2))/(2*delta_x))
    pm = 1 - delta_t*(sigma**2)/(delta_x**2) - r*delta_t
    pd = delta_t * (sigma**2/(2*(delta_x**2)) - (r-0.5*(sigma**2))/(2*delta_x))
    M = round(T/delta_t)
    N = int(round((np.log(SN)-np.log(S0))/delta_x))
    
    X = np.linspace(np.log(SN), np.log(S0), N+1) # ln(S)
    all_S = np.exp(X).reshape(N+1,1) # stock price
    F = np.where(K-all_S>0, K-all_S, 0) # put option values at T
    S = np.repeat(all_S,M+1).reshape(N+1,M+1)
    P = np.repeat(np.array([pu,pm,pd]),N+1).reshape(3, N+1).T
    A = np.c_[P, np.zeros((N-2)*(N+1)).reshape(N+1,N-2)]
    for n in range(2,N):
        A[n] = np.roll(A[n],n-1)
    A[N] = np.roll(A[N],N-2)
    
    for i in np.linspace(M-1,0,M):
        i = int(i)
        B = np.append(np.zeros(N),(S[N-1,i]-S[N,i])).reshape(N+1,1)
        F = np.dot(A,F) + B
    
    return F

In [7]:
# Implicit Finite-Difference
def IFD_put(sigma,r,delta_t,delta_x,K,T,S0,SN):
    
    pu = -0.5 * delta_t * ((sigma**2)/(delta_x**2) + (r-0.5*(sigma**2))/delta_x)
    pm = 1 + delta_t*(sigma**2)/(delta_x**2) + r*delta_t
    pd = -0.5 * delta_t * ((sigma**2)/(delta_x**2) - (r-0.5*(sigma**2))/delta_x)
    M = round(T/delta_t)
    N = int(round((np.log(SN)-np.log(S0))/delta_x))
    
    X = np.linspace(np.log(SN), np.log(S0), N+1) # ln(S)
    all_S = np.exp(X).reshape(N+1,1) # stock price
    F = np.where(K-all_S>0, K-all_S, 0) # put option values at T
    S = np.repeat(all_S,M+1).reshape(N+1,M+1)
    P = np.repeat(np.array([pu,pm,pd]),N+1).reshape(3, N+1).T
    A = np.c_[P, np.zeros((N-2)*(N+1)).reshape(N+1,N-2)]
    for n in range(2,N):
        A[n] = np.roll(A[n],n-1)
    A[0] = np.append([1,-1],np.zeros(N-1))
    A[N] = np.append(np.zeros(N-1),[1,-1])
    
    for i in np.linspace(M-1,0,M):
        i = int(i)
        B = F
        B[0] = 0
        B[N] = S[N,i]-S[N-1,i]
        F = np.dot(inv(A),B)
    
    return F

In [11]:
# Crank-Nicolson Finite-Difference
def CFD_put(sigma,r,delta_t,delta_x,K,T,S0,SN):
    
    pu = -0.25 * delta_t * (sigma**2/delta_x**2+(r-0.5*(sigma**2))/delta_x)
    pm = 1 + delta_t*(sigma**2)/(2*delta_x**2) + r*delta_t/2
    pd = -0.25 * delta_t * (sigma**2/delta_x**2-(r-0.5*(sigma**2))/delta_x)
    M = round(T/delta_t)
    N = int(round((np.log(SN)-np.log(S0))/delta_x))
    
    X = np.linspace(np.log(SN), np.log(S0), N+1) # ln(S)
    all_S = np.exp(X).reshape(N+1,1) # stock price
    F = np.where(K-all_S>0, K-all_S, 0) # put option values at T
    S = np.repeat(all_S,M+1).reshape(N+1,M+1)
    P = np.repeat(np.array([pu,pm,pd]),N+1).reshape(3, N+1).T
    A = np.c_[P, np.zeros((N-2)*(N+1)).reshape(N+1,N-2)]
    for n in range(2,N):
        A[n] = np.roll(A[n],n-1)
    A[0] = np.append([1,-1],np.zeros(N-1))
    A[N] = np.append(np.zeros(N-1),[1,-1])
    
    for i in np.linspace(M-1,0,M):
        i = int(i)
        z = np.repeat(np.array([-pu,-(pm-2),-pd]),N+1).reshape(3, N+1).T
        z = np.c_[z, np.zeros((N-2)*(N+1)).reshape(N+1,N-2)]
        for n in range(2,N):
            z[n] = np.roll(z[n],n-1)
        Z = np.dot(z,F)
        Z[0] = 0
        Z[N] = S[N,i]-S[N-1,i]
        F = np.dot(inv(A),Z)
    
    return F

In [ ]:
# Generalization of Finite-Difference Schemes for Americna call & put
def GFD_American(sigma,r,delta_t,delta_S,K,T,S0,SN,alpha,c_p):
    # c_p = 1 means call option, c_p = -1 means put option
    
    M = round(T/delta_t)
    N = int((SN-S0)/delta_S)
    all_S = np.linspace(SN,S0,N+1)
    j = all_S/delta_S
    ex_values = np.where(c_p*(all_S-K)>0, c_p*(all_S-K), 0).reshape(N+1,1)
    S = np.repeat(all_S,M+1).reshape(N+1,M+1)
    
    # calculate a1,a2,a3,b1,b2,b3
    A = np.zeros((N+1)**2).reshape(N+1,N+1)
    B = np.zeros((N+1)**2).reshape(N+1,N+1)
    A[0,0] = 1
    A[0,1] = -1
    A[N,N-1] = 1
    A[N,N] = -1
    for n in range(1,N):
        a1 = (1-alpha) * ((sigma**2)*(j[n]**2)-r*j[n]) / 2
        a2 = -1/delta_t - ((sigma**2)*(j[n]**2)+r) * (1-alpha)
        a3 = (1-alpha) * ((sigma**2)*(j[n]**2)+r*j[n]) / 2
        b1 = alpha * ((sigma**2)*(j[n]**2)-r*j[n]) / 2
        b2 = 1/delta_t - ((sigma**2)*(j[n]**2)+r) * alpha
        b3 = alpha * ((sigma**2)*(j[n]**2)+r*j[n]) / 2
        A[n,n-1] = a3
        A[n,n] = a2
        A[n,n+1] = a1
        B[n,n-1] = -b3
        B[n,n] = -b2
        B[n,n+1] = -b1
    
    F = ex_values
    for i in np.linspace(M-1,0,M):
        i = int(i)
        d = np.dot(B,F)
        if c_p == 1:
            d[0] = delta_S
            d[N] = 0
        if c_p == -1:
            d[0] = 0
            d[N] = delta_S
        F = np.dot(inv(A),d)
        F = np.where(F >= ex_values, F, ex_values)
        
    return F

# Fixed Income

In [ ]:
# Vasicek model - r
def VasicekPath(dt,T,r0,sigma,k,mean_r):
    r = []
    r.append(r0)
    n = int(round(T/dt))
    for i in range(0,n):
        ri = r[i] + k*(mean_r-r[i])*dt + sigma*np.sqrt(dt)*np.random.normal(0,1,1)[0]
        r.append(ri)
    R = sum(dt*(pd.Series(r))[:n]) # discount rate 
    return R,r

In [ ]:
# Vasicek model - ZCB price formula
def VasicekZCB(T,t,sigma,k,mean_r,rt):
    B = 1/k * (1 - np.exp(-k*(T-t)))
    A = np.exp((mean_r - sigma**2/2/(k**2))*(B-T+t) - (sigma**2)*(B**2)/4/k)
    price = A * np.exp(-B*rt)
    return price

In [ ]:
# CIR - r
def CIRPath(dt,S,r0,sigma,k,mean_r):
    r = []
    r.append(r0)
    n = int(round(S/dt))
    for i in range(0,n):
        ri = r[i] + k*(mean_r-r[i])*dt + sigma*np.sqrt(dt*r[i])*np.random.normal(0,1,1)[0]
        r.append(ri)
    R = sum(dt*(pd.Series(r))[:n]) # discount rate 
    return R,r

In [15]:
# CIR - European Call
def CIREurCall(k,sigma,mean_r,t,S,T,K,M,rt):
    
    h1 = np.sqrt(k**2+2*sigma**2)
    h2 = (k+h1)/2
    h3 = 2*k*mean_r/(sigma**2)
    
    AT_S = ((h1*np.exp(h2*(S-T)))/(h2*(np.exp(h1*(S-T))-1)+h1)) ** h3
    BT_S = (np.exp(h1*(S-T))-1) / (h2*(np.exp(h1*(S-T))-1)+h1)

    theta = np.sqrt(k**2 + 2*(sigma**2))
    phi = (2*theta) / ((sigma**2)*(np.exp(theta*T)-1))
    psi = (k+theta) / (sigma**2)
    r_star = np.log(AT_S/(K/M)) / BT_S

    chi1_x = 2*r_star*(phi+psi+BT_S)
    chi1_p = (4*k*mean_r)/(sigma**2)
    chi1_q = 2*(phi**2)*rt*np.exp(theta*T)/(phi+psi+BT_S)

    chi2_x = 2*r_star*(phi+psi)
    chi2_p = 4*k*mean_r/(sigma**2)
    chi2_q = 2*(phi**2)*rt*np.exp(theta*T)/(phi+psi)

    At_T = ((h1*np.exp(h2*(T-t)))/(h2*(np.exp(h1*(T-t))-1)+h1)) ** h3
    Bt_T = (np.exp(h1*(T-t))-1)/(h2*(np.exp(h1*(T-t))-1)+h1)
    At_S = ((h1*np.exp(h2*(S-t)))/(h2*(np.exp(h1*(S-t))-1)+h1)) ** h3
    Bt_S = (np.exp(h1*(S-t))-1)/(h2*(np.exp(h1*(S-t))-1)+h1)
    Pt_T = At_T*np.exp(-Bt_T*rt) 
    Pt_S = At_S*np.exp(-Bt_S*rt) 

    explicit_price = (Pt_S*ncx2.cdf(chi1_x,chi1_p,chi1_q) - (K/M)*Pt_T*ncx2.cdf(chi2_x,chi2_p,chi2_q))*M
    
    return explicit_price

In [ ]:
# G2++ - r
def G2Path(delta_t,x0,y0,phi,sigma,yita,rho,a,b,period):
    x = []
    y = []
    x.append(x0)
    y.append(y0)
    n = int(round(period/delta_t))
    for i in range(0,n):
        z = np.random.normal(0,1,2)
        w1 = z[0]
        w2 = rho*z[0] + np.sqrt(1-rho**2)*z[1]
        xi = x[i] - a*x[i]*delta_t + sigma*np.sqrt(delta_t)*w1
        yi = y[i] - b*y[i]*delta_t + yita*np.sqrt(delta_t)*w2
        x.append(xi)
        y.append(yi)
    r = pd.Series(x) + pd.Series(y) + phi
    R = sum(delta_t*r[:n]) # discount rate  
    return R,x,y

In [21]:
# G2++ - European Put
def G2EurPut(x0,y0,a,b,sigma,yita,phi,rho,S,T,t,K,M):
    Vt_S = (sigma**2)/(a**2) * ((S-t)+(2/a)*np.exp(-a*(S-t))-np.exp(-2*a*(S-t))/(2*a)-3/(2*a)) +\
           (yita**2)/(b**2) * ((S-t)+(2/b)*np.exp(-b*(S-t))-np.exp(-2*b*(S-t))/(2*b)-3/(2*b)) +\
            2*rho*sigma*yita/(a*b) * ((S-t)+(np.exp(-a*(S-t))-1)/a + (np.exp(-b*(S-t))-1)/b - (np.exp(-(a+b)*(S-t))-1)/(a+b))

    Vt_T = (sigma**2)/(a**2) * ((T-t)+(2/a)*np.exp(-a*(T-t))-np.exp(-2*a*(T-t))/(2*a)-3/(2*a)) +\
           (yita**2)/(b**2) *((T-t)+(2/b)*np.exp(-b*(T-t))-np.exp(-2*b*(T-t))/(2*b)-3/(2*b)) +\
            2*rho*sigma*yita/a/b * ((T-t)+(np.exp(-a*(T-t))-1)/a + (np.exp(-b*(T-t))-1)/b - (np.exp(-(a+b)*(T-t))-1)/(a+b))

    Pt_S = (np.exp(-(S-t)*phi - (1-np.exp(-a*(S-t)))*x0/a - (1-np.exp(-b*(S-t)))*y0/b + 0.5*Vt_S))
    Pt_T = (np.exp(-(T-t)*phi - (1-np.exp(-a*(T-t)))*x0/a - (1-np.exp(-b*(T-t)))*y0/b + 0.5*Vt_T))
    
    Sigma2 = (sigma**2/(2*a**3))*((1-np.exp(-a*(S-T)))**2)*(1-np.exp(-2*a*(T-t))) +\
             (yita**2/(2*b**3))*((1-np.exp(-b*(S-T)))**2)*(1-np.exp(-2*b*(T-t))) +\
             (2*rho*sigma*yita/(a*b*(a+b)))*(1-np.exp(-a*(S-T)))*(1-np.exp(-b*(S-T)))*(1-np.exp(-(a+b)*(T-t)))
    Sigma = np.sqrt(Sigma2)

    d1 = np.log((K/M)*Pt_T/Pt_S)/Sigma - 0.5*Sigma
    d2 = np.log((K/M)*Pt_T/Pt_S)/Sigma + 0.5*Sigma
    price = (-Pt_S*norm.cdf(d1,0,1) + Pt_T*(K/M)*norm.cdf(d2,0,1))*M
    
    return price

# MBS price

In [ ]:
# CIR model
def CIR(dt,T,r0,sigma,k,mean_r):
    r = []
    r.append(r0)
    n = int(round(T/dt))
    w = np.random.normal(0,1,n)
    for i in range(0,n):
        if r[i] >= 0 :
            ri = r[i] + k*(mean_r-r[i])*dt + sigma*np.sqrt(dt*r[i])*w[i]
        else:
            ri = r[i] + k*(mean_r-r[i])*dt # in case negative r pump out
        r.append(ri)
    return r

In [ ]:
# Numerix Prepayment Model
def NumerixPrepayment(R,pre_r10,pre_PV,PV0,t,month):
    month = int(month)
    RIt = 0.28 + 0.14*np.arctan(-8.57+430*(R-pre_r10))
    BUt = 0.3 + 0.7*(pre_PV/PV0)
    SGt = min(1,t/30)
    SYt = [0.94,0.76,0.74,0.95,0.98,0.92,0.98,1.10,1.18,1.22,1.23,0.98][month-1]
    CPRt = RIt * BUt * SGt * SYt
    return CPRt

In [ ]:
def MBS_Price(dt,T,r0,sigma,k,mean_r,R,PV0,M,OAS):
    
    p = []
    IO = []
    PO = []
    
    for m in tqdm(range(0,M)):
        
        rf = CIR(dt,T,r0,sigma,k,mean_r) # simulate rf 
        Rf = (pd.Series(rf)+OAS).cumsum()*dt # discount rate
        n_m = int(round((1/12)/dt)) # how many dt in one month
        r_m = R/12 # mortgage rate per month
        N = T*12
        PV = []
        PV.append(PV0)
        Ct = []
        disc_rate = []
        IOt = []
        POt = []
        
        # Parameters of CIR model
        h1 = np.sqrt(k**2+2*sigma**2)
        h2 = (k+h1)/2
        h3 = 2*k*mean_r/(sigma**2)
        At = ((h1*np.exp(h2*10))/(h2*(np.exp(h1*10)-1)+h1)) ** h3
        Bt = (np.exp(h1*10)-1)/(h2*(np.exp(h1*10)-1)+h1)
        
        for t in range(1,N+1):
            
            # find the month at t
            if t % 12 == 0:
                month = 12
            else:
                month = t % 12
            
            t_yr = t/12 # t in yr unit
            P_bond = At * np.exp(-Bt*rf[n_m*(t-1)]) # price of 10yr bond
            pre_r10 = (-1/10) * np.log(P_bond) # 10 yr risk-free rate
            CPRt = NumerixPrepayment(R,pre_r10,PV[t-1],PV0,t_yr,month) 
            
            PMTt = (PV[t-1] * r_m) / (1-1/((1+r_m)**(N-(t-1)))) # PMT
            IPt = PV[t-1] * r_m # interest payment
            SPt = PMTt - IPt # scheduled pricipal payment
            PPt = (PV[t-1]-SPt) * (1-(1-CPRt)**(1/12)) # prepayment
            
            PV.append(PV[t-1]-SPt-PPt) # PVt
            Ct.append(SPt+PPt+IPt) # cash flow
            IOt.append(IPt) # Interest-only
            POt.append(SPt+PPt) # Principal-only
            disc_rate.append(Rf[n_m*t]) # discount rate
        
        pi = sum(pd.Series(Ct)*np.exp(-1*pd.Series(disc_rate)))
        IOi = sum(pd.Series(IOt)*np.exp(-1*pd.Series(disc_rate)))
        POi = sum(pd.Series(POt)*np.exp(-1*pd.Series(disc_rate)))
        p.append(pi)
        IO.append(IOi)
        PO.append(POi)
        
    return np.mean(p),np.mean(IO),np.mean(PO)